# Langchain 1.0

- langchain 1.0 실습 가이드입니다.
- 0버전과 문법 및 객체 구조가 매우 다르므로 GPT를 통한 코드 작성이 매우 어렵습니다. 반드시 작동이 확인된 예시 코드와 공식 문서를 기반으로 개발하시기 바랍니다.
- 코드 흐름을 따라가면서 이해되지 않는 개념이 있으면 함께 드린 마크다운 파일을 다시 읽어보세요!

## 1. 환경 설정

- Python 3.12.12 기반의 conda 가상 환경에서 테스트되었습니다.
- environment.yml 또는 requirements.txt 파일을 통해 환경을 재현하실 수 있습니다.
- 환경 변수만 잘 등록해준다면 venv, colab 등 다른 실행 환경에서도 작동할 것으로 예상되나 버전 정보를 잘 확인해주세요.

In [2]:
# 공통 라이브러리 import
import os

# 환경 변수 로드
from dotenv import load_dotenv
load_dotenv()

# 예쁜 출력
from pprint import pprint

## 2. 단일 Model 호출

- Agent 없이 단일 Model을 호출하는 방법을 알아봅니다.
- invoke()는 모델 또는 체인을 한 번 실행하여 모델의 응답을 반환받는 메서드입니다.
- Agent와 체인을 만들기 전에, 단일 Model을 호출해봅시다.

In [3]:
# OpenAI 채팅 모델 초기화의 가장 기본적인 형태
from langchain.chat_models import init_chat_model
model = init_chat_model("openai:gpt-5-nano") # api key는 환경 변수에서 자동으로 로드됩니다. api_key 매개변수로 직접 전달할 수도 있습니다.

In [4]:
# invoke 메서드로 독립적인 LLM API를 호출합니다.
response = model.invoke('안녕')

# 모델이 보내준 응답 전체를 출력하면, 토큰 사용량 등 메타 데이터가 함께 출력됩니다.
pprint(response)

AIMessage(content='안녕하세요! 무엇을 도와드릴까요? 궁금한 점 풀이, 글쓰기나 번역, 코딩 문제, 공부 도움 등 어떤 주제든 말씀해 주세요. 시작하고 싶은 주제나 질문이 있나요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 443, 'prompt_tokens': 8, 'total_tokens': 451, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 384, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CXmh5CiBdKAbhP2iJBLJ0Rr8bKIi9', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--47911597-a386-4e4b-a5e0-5d5cc65c251e-0', usage_metadata={'input_tokens': 8, 'output_tokens': 443, 'total_tokens': 451, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 384}})


In [5]:
# 모델이 보내준 실제 답변 내용만 출력하려면 응답의 content 속성을 참조합니다.
pprint(response.content)

('안녕하세요! 무엇을 도와드릴까요? 궁금한 점 풀이, 글쓰기나 번역, 코딩 문제, 공부 도움 등 어떤 주제든 말씀해 주세요. 시작하고 싶은 '
 '주제나 질문이 있나요?')


In [6]:
# 모델 초기화 시 다양한 파라미터를 지정할 수 있습니다.
# OpenAI 모델 외 다양한 모델에 대해서도 동일한 파라미터로 요청을 보낼 수 있습니다.
model = init_chat_model(
    model = "openai:gpt-5-nano",
    temperature=0.8, # 응답의 창의성. GPT-5 계열 모델들은 temperature 파라미터를 지원하지 않지만, Langchain에서는 일관된 인터페이스를 제공하기 때문에 오류가 발생하지 않습니다.
    max_tokens=2000, # 최대 토큰 수
    timeout=20, # 요청 제한 시간(초)
    max_retries=2, # 요청 실패 시 재시도 횟수
    )

In [7]:
# 파라미터가 적용된 모델로도 다시 호출해봅시다.
response = model.invoke('너 웹 서치 기능도 가지고 있니?') # 기본 모델은 웹 서치 기능을 가지고 있지 않습니다.
pprint(response.content)

('상황에 따라 다릅니다. 기본적으로는 웹 검색이 자동으로 활성화되어 있지 않지만, 사용 중인 플랫폼에서 브라우징 기능을 켜면 실시간으로 웹 '
 '검색이 가능합니다.\n'
 '\n'
 '- 웹 검색이 가능하면: "웹 검색으로 찾아줘" 같은 요청으로 최신 정보를 찾아 드립니다(출처도 함께 제공 가능).\n'
 '- 웹 검색이 불가능하면: 제 지식 범위(주로 2024년 6월까지) 내에서 답변하고, 필요한 경우 일반적인 방법이나 출처 확인 방법을 '
 '안내해 드립니다.\n'
 '\n'
 '원하시면 현재 세션에서 웹 검색을 사용할 수 있는지 확인해 드릴까요? 그리고 찾고 싶은 주제나 키워드를 알려주시면 그에 맞춰 '
 '진행하겠습니다.')


## 3. Agent 생성

- 아무 추가 기능 없는 단일 모델 호출하려고 Langchain을 쓰는 건 아니죠.
- Langchain의 핵심 기능인 Agent를 만들어보겠습니다.

In [8]:
# Agent 생성에 필요한 모듈입니다.
from langchain.agents import create_agent

# 앞서 초기화한 Model을 사용하여 Agent를 생성합니다.
agent = create_agent(
    model=model, # 사용할 LLM 모델을 미리 초기화(init)한 후 인자로 전달하세요.
    tools=[] # tool은 필수 인자지만, 빈 리스트를 전달하면 아무 tool도 없는 Agent를 생성할 수 있습니다.
    )

In [9]:
# 모델에 전달할 메시지를 OpenAI의 Chat API 형식에 맞게 작성합니다.
# 다른 모델은 다른 형식을 요구할 수 있으니, 모델 문서를 참고하세요.
messages = [
    {"role": "user", "content": "오늘 서울 날씨는 어때?"},
]

# Agent를 통해 모델을 호출합니다. Agent에 대한 invoke는 모델 invoke와는 다르게 dictionary 형태의 입력을 받으니 주의하세요.
# 검색 tool이 없기 때문에, 실시간 날씨 정보를 제공하지는 못할 것입니다.
response = agent.invoke({"messages":messages})
pprint(response)

{'messages': [HumanMessage(content='오늘 서울 날씨는 어때?', additional_kwargs={}, response_metadata={}, id='68ed3ca3-a1c0-4b2c-9d8b-5861c6b824b0'),
              AIMessage(content='지금은 실시간 날씨 정보를 직접 조회할 수 있는 상태가 아니라 오늘 서울 날씨를 바로 알려드리진 못해요. 원하시면 인터넷에서 현재 날씨를 찾아 알려드릴게요.\n\n검색해 드릴까요? 원하신다면 다음 정보를 같이 알려드리겠습니다.\n- 현재 기온\n- 강수확률\n- 강수 유형(비/눈/빗방울)\n- 바람 세기/ 방향\n\n또는 간단한 팁이 필요하시면 오늘의 일반적인 외출 팁도 드릴 수 있어요. 어떤 방식으로 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 1101, 'prompt_tokens': 14, 'total_tokens': 1115, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 960, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CXmhNfbxQjFMRlAyvcAlAa2zcmfqx', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--c486a49b-dc18-

In [10]:
# 실제 답변 내용만 출력하려면 messages 리스트의 마지막 요소의 content 속성을 참조합니다.
pprint(response["messages"][-1].content)

('지금은 실시간 날씨 정보를 직접 조회할 수 있는 상태가 아니라 오늘 서울 날씨를 바로 알려드리진 못해요. 원하시면 인터넷에서 현재 날씨를 '
 '찾아 알려드릴게요.\n'
 '\n'
 '검색해 드릴까요? 원하신다면 다음 정보를 같이 알려드리겠습니다.\n'
 '- 현재 기온\n'
 '- 강수확률\n'
 '- 강수 유형(비/눈/빗방울)\n'
 '- 바람 세기/ 방향\n'
 '\n'
 '또는 간단한 팁이 필요하시면 오늘의 일반적인 외출 팁도 드릴 수 있어요. 어떤 방식으로 도와드릴까요?')


In [11]:
# Agent 생성 시에도 다양한 파라미터를 지정할 수 있습니다.
# 오늘 다 다뤄보지는 못하겠지만, Agent의 세부 동작은 모두 이 파라미터들로 조정할 수 있습니다.
# 아래 있는 것들 외에도 더 많은 파라미터가 있으니 공식 문서를 참고하세요.
agent = create_agent(
    model=model,
    tools=[], # tool 리스트는 다음 챕터에서 다뤄보겠습니다.
    system_prompt="You are a helpful assistant.", # Agent의 시스템 프롬프트입니다.
    middleware=[], # Agent의 동작 사이에 발생하는 요청이나 응답을 가로채서 처리하는 고급 기능입니다.
)

## 3. tool 정의

- 위에서 만든 Agent는 별다른 기능이 없으니 단일 모델 호출과 별로 다를 게 없죠.
- 이제 Agent가 사용할 tool을 만들어 봅시다.
- tool의 종류를 세 가지로 나누어 각각 만들어보겠습니다.

### 3-1. 커스텀 tool 정의

- 직접 만든 Python 함수를 tool로 사용할 수 있습니다.
- URL 또는 라이브러리를 통해 API를 호출하여 응답을 반환하는 tool도 이 방법으로 구현할 수 있습니다.

In [12]:
# tool 정의를 위해 필요한 모듈입니다.
from langchain.tools import tool

# 커스텀 tool 생성
@tool # 데코레이터를 사용하여 tool로 등록합니다.
def calculator(num_1:int, num_2:int) -> int: # typehint는 Agent가 tool의 입출력 형식을 이해하는 데 도움을 줍니다. 안정적인 작동을 위해 반드시 작성하는게 좋습니다.
    """입력받은 두 수의 덧셈을 반환합니다.""" # docstring은 tool의 설명으로 사용됩니다. Agent가 tool을 선택하는 데 도움을 줍니다.
    return num_1 + num_2

### 3-2. langchain-community에 통합된 외부 tool 사용

- 라이브러리를 통해 제공되는 외부 tool들을 langchain-community에서 통합하여 사용할 수 있습니다.
- 웬만한 기능들은 langchain 생태계 내에서 쉽게 쓸 수 있으니 목록을 확인하고 활용해봅시다.
- https://docs.langchain.com/oss/python/integrations/providers/overview

In [15]:
# langchain community에서 통합되어 있는 외부 tool의 목록을 확인해봅시다.
import importlib, pkgutil

package = importlib.import_module("langchain_community.tools")

for mod in pkgutil.iter_modules(package.__path__):
    print(mod.name)

ainetwork
amadeus
arxiv
asknews
audio
azure_ai_services
azure_cognitive_services
bearly
bing_search
brave_search
cassandra_database
clickup
cogniswitch
connery
convert_to_openai
databricks
dataforseo_api_search
dataherald
ddg_search
e2b_data_analysis
edenai
eleven_labs
few_shot
file_management
financial_datasets
github
gitlab
gmail
golden_query
google_books
google_cloud
google_finance
google_jobs
google_lens
google_places
google_scholar
google_search
google_serper
google_trends
graphql
human
ifttt
interaction
jina_search
jira
json
memorize
merriam_webster
metaphor_search
mojeek_search
multion
nasa
nuclia
office365
openai_dalle_image_generation
openapi
openweathermap
passio_nutrition_ai
playwright
plugin
polygon
powerbi
pubmed
render
requests
riza
scenexplain
searchapi
searx_search
semanticscholar
shell
slack
sleep
spark_sql
sql_database
stackexchange
steam
steamship_image_generation
tavily_search
vectorstore
wikidata
wikipedia
wolfram_alpha
yahoo_finance_news
you
youtube
zapier
zenguar

In [16]:
# duckduckgo는 api key 없이도 사용할 수 있는 웹 검색 도구입니다.
import langchain_community.tools.ddg_search

# duckduckgo 검색 tool의 속성을 확인하여 검색 기능 호출 방법을 알아봅시다.
print(dir(langchain_community.tools.ddg_search))

['DuckDuckGoSearchRun', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'tool']


In [18]:
# 위에서 확인한 duckduckgo의 검색 기능을 import합니다.
from langchain_community.tools.ddg_search import DuckDuckGoSearchRun

# 검색 tool 인스턴스를 생성합니다.
ddg_search_tool = DuckDuckGoSearchRun()

In [19]:
# 세 번째 방법을 배우기 전에, 위에서 만든 두 가지 tool을 Agent에게 전달해봅시다.
# nano 모델은 Agent로 사용하기엔 너무 성능이 낮아 무한 루프 등 의도하지 않은 동작이 발생할 수 있으므로 mini 모델로 변경하겠습니다.
model = init_chat_model("openai:gpt-5-mini")

# agent 생성 및 tool 전달
agent = create_agent(
    model=model,
    tools=[calculator, ddg_search_tool], # 앞서 만든 두 가지 tool을 리스트로 전달합니다.
)

In [20]:
# Agent에 전달할 메시지 객체를 생성하기 위해 필요한 모듈입니다.
from langchain.messages import HumanMessage

# Agent에 전달할 메시지 객체를 생성합니다.
messages = [
    HumanMessage(content="2 더하기 3은 얼마야? 그리고 오늘 서울 날씨는 어때?"),
]

# Agent에 대한 invoke는 모델 invoke와는 다르게 dictionary 형태의 입력을 받으니 주의하세요.
response = agent.invoke({"messages":messages})


# Agent가 보내준 전체 응답에는 추론 과정과 tool 사용 내역이 포함되어 있습니다.
pprint(response)

{'messages': [HumanMessage(content='2 더하기 3은 얼마야? 그리고 오늘 서울 날씨는 어때?', additional_kwargs={}, response_metadata={}, id='4e50331f-c479-43ee-9ceb-8dd97768e2fc'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 259, 'prompt_tokens': 207, 'total_tokens': 466, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 192, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CXmlh5PQsDctauxKmSLuiaGu1yzA4', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--42b421b0-6a7e-4a04-b129-f70b6a658e77-0', tool_calls=[{'name': 'calculator', 'args': {'num_1': 2, 'num_2': 3}, 'id': 'call_nSfkHuKvnxblNurVOoL8cOs0', 'type': 'tool_call'}, {'name': 'duckduckgo_search', 'args': {'query': '오늘 서울

In [21]:
# 실제 답변 내용만 출력하려면 messages 리스트의 마지막 요소의 content 속성을 참조합니다.
pprint(response["messages"][-1].content)

('2 더하기 3 = 5.\n'
 '\n'
 '오늘 서울 날씨는 지금 바로 실시간으로 확인해 드릴까요? (현재 기온·체감·습도·강수확률·풍속·미세먼지 중 어떤 정보를 원하시는지도 '
 '알려주시면 더 정확히 알려드릴게요.)')


### 3-3. 모델 vendor들이 제공하는 tool을 모델에 바인딩

#### **※ 주의!! ※**
- 현재 버전에서 Agent를 통하지 않고 단일 Model을 직접 호출해야만 사용할 수 있는 기능입니다.
- Agent를 활용하는 경우 사용할 수 없는 방법이므로 헷갈리지 마세요!
- 설계가 복잡해지고 세밀한 구조 설계, 유지보수를 어렵게 만드는 방식이니 사용하지 마세요.
- 다만, 이 방법을 아예 모르고 있으면 GPT나 Docs 보고 코드 쓰다가 tool 정의 및 호출 방법을 섞어서 쓰다가 코드가 꼬일 수 있기 때문에 개념만 이해하고 넘어갑시다.
---
#### 실습
- LLM API를 제공하는 vendor들은 모델에서 곧바로 이용할 수 있는 tool을 함께 서비스하고 있습니다.
- 예를 들어, OpenAI의 경우 웹 서치, MCP 서버 연결, 벡터 스토어 검색, 코드 실행 등의 도구를 제공합니다.
- vendor가 제공하는 tool을 Agent에 바인딩하여 사용해보겠습니다.

In [22]:
from langchain_openai import ChatOpenAI

# 객체 생성
llm = ChatOpenAI(
    temperature=0.5,
    model_name="gpt-5-mini",  # 모델명
)

# 질의내용
question = "대한민국의 수도는 어디인가요?"

# 질의
print(f"[답변]: {llm.invoke(question)}")

[답변]: content='대한민국의 수도는 서울특별시(서울)입니다.  \n참고로 정부의 일부 행정 기능과 부처는 세종특별자치시로 이전되어 있어 주요 행정업무는 세종에도 분산되어 있습니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 190, 'prompt_tokens': 15, 'total_tokens': 205, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 128, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CXmmMxNr6v6Rh4HXnqWiUYuSCN7GC', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--66750ee1-9b9c-4cee-9387-cb3c3d04b1d1-0' usage_metadata={'input_tokens': 15, 'output_tokens': 190, 'total_tokens': 205, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 128}}


In [ ]:
# 모델 vendor들이 제공하는 tool을 모델에 바인딩하기 위해 필요한 모듈입니다.
from langchain_openai import ChatOpenAI

# OpenAI 전용 Model 초기화 메서드입니다.
# OpenAI 모델에서 제공하는 tool을 바인딩하기 위해서는 모델 초기화시 ChatOpenAI 클래스를 사용해야 합니다.
model = ChatOpenAI(
    model="gpt-5-mini",
)

# ChatOpenAI 모델 인스턴스에 커스텀 tool을 바인딩합니다.


# 모델이 사용할 tool 리스트를 정의합니다.
# 딕셔너리 형태의 web search tool은 openai가 제공하는 tool이고, calculator는 위에서 직접 만든 커스텀 tool입니다.
# ddg는 웹 검색 역할이 중복되므로 제외합니다.
tools = [{"type": "web_search"}, calculator]

model_with_calculator = model.bind_tools(tools) 

# model을 호출합니다.
response = model_with_calculator.invoke(
    "올해 11월에 한국에서 개봉하는 주요 영화 3개만 알려줘. 그리고 12311 더하기 112455는 몇이야?",
)

# 추론 과정에서 커스텀 tool이 호출되면, 추론 루프를 멈추고 AIMessage를 반환합니다.
# Agent가 아닌 단일 모델에 대한 invoke이기 때문에 커스텀 tool 호출에는 대응하지 못하기 때문입니다.
# 별도로 다시 invoke를 체이닝하는 등 추가 코드가 필요하나, 너무 구린 방식이니까 그냥 넘어가겠습니다.
pprint(response)

AIMessage(content=[{'id': 'rs_03dfaa575fe18cdc00690894d59ce8819db1c8cffd846dc9b3', 'summary': [], 'type': 'reasoning'}, {'id': 'ws_03dfaa575fe18cdc00690894df48c4819d99d669766fa44765', 'action': {'query': '2025년 11월 한국 개봉 영화 주요작', 'type': 'search'}, 'status': 'completed', 'type': 'web_search_call'}, {'id': 'rs_03dfaa575fe18cdc00690894e0a020819dad5b01cf1be98645', 'summary': [], 'type': 'reasoning'}, {'id': 'ws_03dfaa575fe18cdc00690894e1d6b8819da046b52506c7ee22', 'action': {'query': '2025년 11월 개봉작 목록 한국', 'type': 'search'}, 'status': 'completed', 'type': 'web_search_call'}, {'id': 'rs_03dfaa575fe18cdc00690894e34e10819dbfec805eac8089f5', 'summary': [], 'type': 'reasoning'}, {'id': 'ws_03dfaa575fe18cdc00690894e5b0b0819d954cc574bdeb197c', 'action': {'query': "Wicked: Part Two Korea release November 19 2025 'Wicked: The Wide World' 'Wicked: Part 2' Korea release 2025", 'type': 'search'}, 'status': 'completed', 'type': 'web_search_call'}, {'id': 'rs_03dfaa575fe18cdc00690894e7d3e4819dbabb5b90e2

In [ ]:
# # 이번에도 실제 답변 내용만 출력해봅시다. 조금 더 깔끔한 출력을 위해 content의 각 파트를 순회하며 줄 단위로 출력합니다.
# for part in response.content:
#     if part.get("type") == "text":
#         print(part["text"])

## 4. 대화 기록 저장
